# Harmonic oscillator

The second order differential equation describing the harmonic oscillator is given by:
$m \frac{d^2 \theta}{dt^2}(t) = -\frac{g}{l}\theta(t) - q\frac{d\theta}{dt}(t) + F_d \sin \Omega_d t$. To solve it, this equation is rewritten as a set of two first order differential equations by introducing $\omega(t) = \frac{d\theta}{dt}(t)$.

## Preparation

Load the required modules.

In [1]:
from bokeh.io import output_notebook, push_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, PanTool, ResetTool, WheelZoomTool
from bokeh.plotting import figure
from ipywidgets import interact
import numpy as np
from scipy.integrate import ode
output_notebook()

Loading BokehJS ...

## Definitions

Implement the function that define the ordinary first order differential equations to solve.

In [2]:
def func(t, y, l, q, F_d, omega_d):
    g = 9.81
    return [
        y[1],
        -(g/l)*y[0] - q*y[1] + F_d*np.sin(omega_d*t)
    ]

The Jacobian is given by the matrix defined as follows.

In [3]:
def jacobian(t, y, l, q, F_d, omega_d):
    g = 9.81
    return [
        [0.0, 1.0],
        [-g/l, -q]
    ]

Define a function to solve the set of differential equations for the giving initial conditions, and parameters. The result is a dictionary.

In [4]:
def solve(func, jac, t_max=20.0, delta_t=0.01,
          theta_0=0.1, omega_0=0.0, params={'l': 9.81, 'q': 0.0,
                                            'F_d': 0.0, 'Omega_d': 0.5},
          atol=1.0e-6, rtol=0.0):
    t_0 = 0.0
    integrator = ode(func, jac).set_integrator('dopri', atol=atol, rtol=rtol)
    integrator.set_initial_value([theta_0, omega_0], t_0)
    integrator.set_f_params(params['l'], params['q'], params['F_d'], params['Omega_d'])
    integrator.set_jac_params(params['l'], params['q'], params['F_d'], params['Omega_d'])
    times = [t_0]
    thetas = [theta_0]
    omegas = [omega_0]
    while integrator.successful() and integrator.t < t_max:
        integrator.integrate(integrator.t + delta_t)
        times.append(integrator.t)
        theta = integrator.y[0]
        while theta > np.pi:
            theta -= 2.0*np.pi
        while theta < -np.pi:
            theta += 2.0*np.pi
        thetas.append(theta)
        omegas.append(integrator.y[1])
    return {'t': times, 'theta': thetas, 'omega': omegas}

## Interactive parameters

Compute the initial plot for a harmonic, non-damped, non-driven pendulum.

In [18]:
data = solve(func, jacobian)

Create the original plots for $\theta(t)$, $\omega(t)$, and $\omega(\theta)$.

In [19]:
plot_options = {'plot_width': 250, 'plot_height': 200,
                'tools': 'pan,wheel_zoom,help'}
theta_fig = figure(**plot_options)
theta_fig.xaxis.axis_label = 't'
theta_fig.yaxis.axis_label = 'theta'
theta_plot = theta_fig.line(data['t'], data['theta'])
omega_fig = figure(x_range=theta_fig.x_range, y_range=theta_fig.y_range,
                   **plot_options)
omega_fig.xaxis.axis_label = 't'
omega_fig.yaxis.axis_label = 'omega'
omega_plot = omega_fig.line(data['t'], data['omega'])
phase_fig = figure(x_range=theta_fig.y_range, y_range=omega_fig.y_range,
                   **plot_options)
phase_fig.xaxis.axis_label = 'theta'
phase_fig.yaxis.axis_label = 'omega'
phase_plot = phase_fig.line(data['theta'], data['omega'])
plot = gridplot([[theta_fig, omega_fig], [None, phase_fig]]);

Define the update function with the appropriate parameters.

In [20]:
def update(l=9.81, q=0.0, F_d=0.0, Omega_d=0.5, t_max=20.0):
    params = {'l': l, 'q': q, 'F_d': F_d, 'Omega_d': Omega_d}
    results = solve(func, jacobian, t_max=t_max, params=params)
    theta_plot.data_source.data['x'] = results['t']
    theta_plot.data_source.data['y'] = results['theta']
    omega_plot.data_source.data['x'] = results['t']
    omega_plot.data_source.data['y'] = results['omega']
    phase_plot.data_source.data['x'] = results['theta']
    phase_plot.data_source.data['y'] = results['omega']
    push_notebook()

Show the plots, and the widgets for interaction.

In [24]:
show(plot, notebook_handle=True);

In [22]:
interact(update, l=(5.0, 15.0), q=(0.0, 5.0), F_d=(0.0, 2.0), Omega_d=(0.1, 1.5), t_max=(20.0, 100.0));

interactive(children=(FloatSlider(value=9.81, description='l', max=15.0, min=5.0), FloatSlider(value=0.0, desc…

## Exploring the phase space

Create scatter plots for linked brushing so that the selection in one plot is shown in all others. This makes interpreting the phase plot easier.

In [15]:
params = {'l': 9.81, 'q': 0.2, 'F_d': 0.4, 'Omega_d': 0.5}
data = solve(func, jacobian, params=params, t_max=100.0)
stride=50
point_source = ColumnDataSource(data={'t': data['t'][::stride],
                                      'theta': data['theta'][::stride],
                                      'omega': data['omega'][::stride]})

In [16]:
plot_options = {'plot_width': 350, 'plot_height': 300,
                'tools': 'pan,wheel_zoom,box_select,help'}
theta_fig = figure(**plot_options)
theta_fig.xaxis.axis_label = 't'
theta_fig.yaxis.axis_label = 'theta'
theta_plot = theta_fig.circle('t', 'theta', source=point_source)
omega_fig = figure(x_range=theta_fig.x_range, y_range=theta_fig.y_range,
                   **plot_options)
omega_fig.xaxis.axis_label = 't'
omega_fig.yaxis.axis_label = 'omega'
omega_plot = omega_fig.circle('t', 'omega', source=point_source)
phase_fig = figure(x_range=theta_fig.y_range, y_range=omega_fig.y_range,
                   **plot_options)
phase_fig.xaxis.axis_label = 'theta'
phase_fig.yaxis.axis_label = 'omega'
phase_plot = phase_fig.circle('theta', 'omega', source=point_source)
plot = gridplot([[theta_fig, omega_fig], [None, phase_fig]]);
show(plot)

Create a scatter plot of the phase space that shows $t$, $\theta(t)$, and $\omega(t)$ when hovering over a point.

In [17]:
hover_tool = HoverTool(tooltips=[('(t, theta, omega)', '(@t, @theta, @omega)')])
phase_fig = figure(plot_width=500, plot_height=300, tools=[PanTool(), WheelZoomTool(), hover_tool, ResetTool()])
phase_fig.xaxis.axis_label = 'theta'
phase_fig.yaxis.axis_label = 'omega'
phase_plot = phase_fig.circle('theta', 'omega', source=point_source)
show(phase_fig)